# Metro transit information 

Using the metro API, this notebook gets data on travel times to Hattie Mae White on board meeting days. 

In [11]:
import numpy as np
import pandas as pd
import requests
import time
import dateutil

direc = "//Users//afan//Desktop//Misc//HMW_Transit//"

## Testing things

It's been a while since I've worked with an API, so let's just make sure some things are right

In [4]:
# Let's get all the information on the one route I ride the most often
response = requests.get("https://api.ridemetro.org/data/Routes?$filter=RouteName eq '004'&subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)

200


In [5]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Routes',
 'value': [{'RouteId': 'Ho414_4620_41894',
   'AgencyAbbreviation': 'RideMetro',
   'FinalStop0Id': None,
   'FinalStop1Id': None,
   'RouteName': '004',
   'LongName': 'Beechnut',
   'RouteType': 'Bus',
   'DataSource': 'Transit_5'}]}

In [6]:
# Now let's get all the information we can about this route. This is literally all the stops! 
response = requests.get("https://api.ridemetro.org/data/Routes('Ho414_4620_41894')?subscription-key=4e7902d1a2ae42df978fe20387049854&$expand=Stops")
print(response.status_code)

200


In [7]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Routes(Stops())',
 'value': [{'RouteId': 'Ho414_4620_41894',
   'AgencyAbbreviation': 'RideMetro',
   'FinalStop0Id': None,
   'FinalStop1Id': None,
   'RouteName': '004',
   'LongName': 'Beechnut',
   'RouteType': 'Bus',
   'DataSource': 'Transit_5',
   'Stops': [{'Id': 'Ho414_4620_94_D_Ho414_4620_41894_0',
     'StopId': 'Ho414_4620_94_D',
     'DirectionId': 'Ho414_4620_41894_0',
     'RouteId': 'Ho414_4620_41894',
     'AgencyId': 'Ho414',
     'Name': 'MISSION BEND TRANSIT CENTER BAY D',
     'StopCode': '94',
     'Type': 'Bus',
     'Lat': 29.711304,
     'Lon': -95.62981,
     'Ordinal': 1,
     'OrdinalOnDirection': 1},
    {'Id': 'Ho414_4620_12323_Ho414_4620_41894_0',
     'StopId': 'Ho414_4620_12323',
     'DirectionId': 'Ho414_4620_41894_0',
     'RouteId': 'Ho414_4620_41894',
     'AgencyId': 'Ho414',
     'Name': 'Howell Sugarland Rd @ Rio Bonito Rd',
     'StopCode': '12323',
     'Type': 'Bus',
    

In [ ]:
# Let's get all the vehicles
response = requests.get("https://api.ridemetro.org/data/Vehicles?subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)


200


In [ ]:
response.json()

## Exploring bus stops around Hattie Mae White

Let's start with how accessible Hattie Mae White is by public transit: Let's see what stops are in the area

In [17]:
# Hattie Mae White coordinatess

HMW_lat = 29.802759908899148
HMW_long = -95.45410037006431

key = "4e7902d1a2ae42df978fe20387049854"

In [18]:
HMW_stops = requests.get(f"https://api.ridemetro.org/data/GeoAreas('{HMW_lat}|{HMW_long}|0.3')/Stops?subscription-key={key}")
print(HMW_stops.status_code)

200


In [19]:
HMW_stops.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Stops',
 'value': [{'StopId': 'Ho414_4620_8412',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Mangum Rd',
   'StopCode': '8412',
   'Type': None,
   'DataSource': '',
   'Lat': 29.801395,
   'Lon': -95.455853,
   'DistanceFromCenter': 0.19903016518379862},
  {'StopId': 'Ho414_4620_8413',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Northwest Freeway',
   'StopCode': '8413',
   'Type': None,
   'DataSource': '',
   'Lat': 29.801401,
   'Lon': -95.45297,
   'DistanceFromCenter': 0.16143556708060636},
  {'StopId': 'Ho414_4620_12077',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Northwest Fwy',
   'StopCode': '12077',
   'Type': None,
   'DataSource': '',
   'Lat': 29.80163,
   'Lon': -95.454342,
   'DistanceFromCenter': 0.09251021044187172},
  {'StopId': 'Ho414_4620_11053',
   'AgencyId': 'Ho414',
   'Name': 'Hempstead Rd @ Post Oak Rd',
   'StopCode': '11053',
   'Type': None,
   'DataSource': '',
   'Lat': 

## Bringing in school information 

In the Google colab version, we imported the data from airtable. We're not going to do that here. Let's just read the file in as a csv

In [14]:
sch_str = direc + "data//school_info.csv"

# select only the columns we want
columns = ['School Name', 'Latitude [Public School] 2019-20', 'Longitude [Public School] 2019-20', 'School ID - NCES Assigned [Public School] Latest available year']
sch_info = pd.read_csv(sch_str, usecols= columns)
sch_info.columns = ['name', 'lat', 'long', 'nces_id']
sch_info.head()

,name,lat,long,nces_id
0,ALCOTT EL,29.667360,-95.329781,482364002385
1,ALMEDA EL,29.600293,-95.418075,482364002387
2,ANDERSON EL,29.648928,-95.487132,482364002388
3,ARABIC IMMERSION MAGNET SCHOOL,29.737146,-95.388448,482364013165
4,ASHFORD EL,29.746776,-95.597539,482364002389


In [15]:
# now, let's take a sample school and calculat the route! 
nrow = 2

s_lat = sch_info.iloc[nrow, 1]
s_long = sch_info.iloc[nrow, 2]

print(sch_info.iloc[nrow, 0])
print(s_lat, s_long)

29.67397345835996, -95.46738418104776


ANDERSON EL
29.648928 -95.487132


(29.67397345835996, -95.46738418104776)

In [20]:
trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")
print(trip.status_code)

200


In [23]:
trip = pd.DataFrame(trip.json())

In [25]:
trip_df = trip['value'].apply(pd.Series)
trip_df['school'] = sch_info.iloc[nrow, 0]
trip_df['school_id'] = sch_info.iloc[nrow, 3]
trip_df.head()

,ItineraryId,Created,Kml,StartTime,EndTime,AdjustedStartTime,AdjustedEndTime,StartStopName,EndStopName,StartStopId,...,WalkDistance,RequestId,StartLon,StartLat,EndLon,EndLat,LocalTimeOffset,Legs,school,school_id
0,9ae13f42-bbc5-4af0-aaa8-50457e0a5c2a,2021-11-20T01:12:38.1982353Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-20T01:46:37Z,2021-11-20T03:14:48Z,2021-11-20T01:46:37Z,2021-11-20T03:14:48Z,Hillcroft Ave @ Ludington Dr,Hempstead Rd @ Post Oak Rd,Ho414_4620_6337,...,0.864935,96580a51-a9d4-484a-b20e-b4e95ba0ac8b,-95.487132,29.648928,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1496890132, 'AgencyAbbrevi...",ANDERSON EL,482364002388
1,198008bb-2449-495a-957a-50e41bc02285,2021-11-20T01:12:38.1983586Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-20T01:34:43Z,2021-11-20T03:18:18Z,2021-11-20T01:34:43Z,2021-11-20T03:18:18Z,W Airport Blvd @ Landsdowne Dr,W 18Th St @ Northwest Freeway,Ho414_4620_6284,...,0.566660,96580a51-a9d4-484a-b20e-b4e95ba0ac8b,-95.487132,29.648928,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1496891132, 'AgencyAbbrevi...",ANDERSON EL,482364002388
2,6f2bf202-516d-4a83-94be-4be5203538dd,2021-11-20T01:12:38.1985365Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-20T01:16:47Z,2021-11-20T03:29:31Z,2021-11-20T01:16:47Z,2021-11-20T03:29:31Z,W Bellfort Ave @ Mullins Dr,Hempstead Rd @ Post Oak Rd,Ho414_4620_6890,...,1.214386,96580a51-a9d4-484a-b20e-b4e95ba0ac8b,-95.487132,29.648928,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1496892132, 'AgencyAbbrevi...",ANDERSON EL,482364002388
3,48b4f8e1-9b7c-4204-a99a-9625cb9d43f8,2021-11-20T01:12:38.1985938Z,http://api.transitiq.com/ODataService.svc/GetI...,2021-11-20T01:16:47Z,2021-11-20T03:33:18Z,2021-11-20T01:16:47Z,2021-11-20T03:33:18Z,W Bellfort Ave @ Mullins Dr,W 18Th St @ Northwest Freeway,Ho414_4620_6890,...,0.968553,96580a51-a9d4-484a-b20e-b4e95ba0ac8b,-95.487132,29.648928,-95.4541,29.80276,-5,"[{'ItineraryLegId': 1496893132, 'AgencyAbbrevi...",ANDERSON EL,482364002388


In [ ]:
legs = trip_df['Legs'].apply(pd.Series)

In [ ]:
legs.iloc[0].apply(pd.Series)

,ItineraryLegId,AgencyAbbreviation,AgencyName,Created,StopName,StopId,RouteId,RouteName,DirectionId,DirectionVariantId,DirectionName,RouteDestinationName,StartTime,AdjustedStartTime,Duration,Length,Address,ShapeUri,TransportType,Ordinal,ItineraryId,DataSource,Lat,Lon,TripId,TripHeadsign,DelayFromScheduleInSeconds
0,1108178896,None,None,2021-11-19T20:54:17.6687139Z,,,,,,,,,2021-11-19T21:31:29Z,2021-11-19T21:31:29Z,979,0.843312,,,Walk,0,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.620428,-95.455033,,,NaN
1,1108178897,RideMetro,Houston,2021-11-19T20:54:17.6687662Z,S Post Oak Rd @ Heatherbrook Dr,Ho414_4620_5256,Ho414_4620_41922,049,Ho414_4620_41922_0,Ho414_4620_41922_0_1,NORTHWEST TC,NORTHWEST TC,2021-11-19T21:48:48Z,2021-11-19T21:48:48Z,4152,11.086886,,,Bus,1,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.622834,-95.464965,Ho414_4620_9267310,NORTHWEST TC,0.0
2,1108178898,RideMetro,Houston,2021-11-19T20:54:17.668788Z,79 - NORTHWEST BAY M,Ho414_4620_79_M,Ho414_4620_41928,058,Ho414_4620_41928_1,Ho414_4620_41928_1_0,WEST BELT,WEST BELT,2021-11-19T23:00:00Z,2021-11-19T23:00:00Z,360,1.104310,,,Bus,3,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.783070,-95.455460,Ho414_4620_9269230,WEST BELT,0.0
3,1108178899,None,None,2021-11-19T20:54:17.6687885Z,Hempstead Rd @ Post Oak Rd,Ho414_4620_11053,,,,,,,2021-11-19T23:07:00Z,2021-11-19T23:07:00Z,468,0.403830,,,Walk,4,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.799026,-95.456516,,,NaN
4,1108178900,None,None,2021-11-19T20:54:17.6687885Z,,,,,,,,,2021-11-19T23:07:00Z,2021-11-19T23:07:00Z,468,0.403830,,,Arrived,5,4991ed22-ce9f-40cb-9c29-d7b5fdf5c387,None,29.802760,-95.454100,,,NaN


In [ ]:
schools.head(20)

,name,lat,long,id
0,GRISSOM EL,29.620428,-95.455033,482364002472
1,KING EARLY CHILDHOOD CTR,29.614025,-95.436000,482364010746
2,KOLTER EL,29.712887,-95.455950,482364002510
3,HOBBY EL,29.635314,-95.436981,482364002490
4,WELCH MIDDLE,29.653218,-95.529156,482364005501
5,HARRIS J R EL,29.718712,-95.277378,482364002478
6,BELLAIRE H S,29.692163,-95.469159,482364002397
7,MADISON H S,29.629169,-95.435617,482364002530
8,WOODSON SCHOOL,29.643053,-95.366681,482364002417
9,MICKEY LELAND COLLEGE PREP ACAD FOR YOUNG MEN,29.776713,-95.335360,482364012618


## Running it on all the schools 

Now, let's loop through all the schools and put all the routes together 

In [ ]:
routelist = []

for nrow in range(0, len(schools)): 

  s_lat = schools.iloc[nrow, 1]
  s_long = schools.iloc[nrow, 2]

  # get the trip information for that school
  trip = requests.get(f"https://api.ridemetro.org/data/CalculateItineraryByPoints?lat1={s_lat}&lon1={s_long}&lat2={HMW_lat}&lon2={HMW_long}&$orderby=EndTime&$expand=Legs&subscription-key={key}")

  trip1 = pd.DataFrame(trip.json())

  trip_df = trip1['value'].apply(pd.Series)
  trip_df['school'] = schools.iloc[nrow, 0]
  trip_df['school_id'] = schools.iloc[nrow, 3]

  routelist.append(trip_df)
  print(f"Done with school #{nrow}: {schools.iloc[nrow, 0]}")

  time.sleep(2)   

Done with school #0
Done with school #1
Done with school #2
Done with school #3
Done with school #4
Done with school #5
Done with school #6
Done with school #7
Done with school #8
Done with school #9
Done with school #10
Done with school #11
Done with school #12
Done with school #13
Done with school #14
Done with school #15
Done with school #16
Done with school #17
Done with school #18
Done with school #19
Done with school #20
Done with school #21
Done with school #22
Done with school #23
Done with school #24
Done with school #25
Done with school #26
Done with school #27
Done with school #28
Done with school #29
Done with school #30
Done with school #31
Done with school #32
Done with school #33
Done with school #34
Done with school #35
Done with school #36
Done with school #37
Done with school #38
Done with school #39
Done with school #40
Done with school #41
Done with school #42
Done with school #43
Done with school #44
Done with school #45
Done with school #46
Done with school #47
Do

In [373]:
routes = pd.concat(routelist, ignore_index = True)
routes['AdjustedStartTime'] = routes.AdjustedStartTime.apply(dateutil.parser.isoparse)
routes['AdjustedEndTime'] = routes.AdjustedEndTime.apply(dateutil.parser.isoparse)
routes['length'] = routes['AdjustedEndTime'] - routes['AdjustedStartTime']

all_legs = routes['Legs'].apply(pd.Series)
leglist = []

for route in range(0, len(all_legs)): 

  temp = all_legs.iloc[route, :].apply(pd.Series)
  leglist.append(temp)

legs = pd.concat(leglist)
print(legs.shape)
legs = legs[~np.isnan(legs['Ordinal'])]
legs.shape

(12410, 28)


(8073, 28)

In [337]:
print(f"Number of schools: {len(schools)}")
print(f"Number of routes: {len(routes)}")
print(f"Number of legs: {len(legs)}")

Number of schools: 280
Number of routes: 1241
Number of legs: 8073


In [ ]:
legs.sort_values(['ItineraryId', 'Ordinal'], inplace = True)
legs.head()

In [377]:
mintrans = routes[routes.TravelTypes != "Walk"].sort_values('TransferCount').drop_duplicates(subset = 'school_id', keep = 'first')
mintrans.TransferCount.value_counts()

2    164
3     79
1     35
4      1
Name: TransferCount, dtype: int64

In [378]:
mintrans.length.describe()

count                          279
mean     0 days 01:20:12.820788530
std      0 days 00:28:27.858972493
min                0 days 00:15:58
25%                0 days 01:01:52
50%                0 days 01:18:38
75%         0 days 01:37:44.500000
max                0 days 03:15:26
Name: length, dtype: object

In [376]:
routes.length.describe()

count                         1241
mean     0 days 01:29:57.339242546
std      0 days 00:29:12.139752715
min                0 days 00:00:31
25%                0 days 01:10:36
50%                0 days 01:27:10
75%                0 days 01:49:22
max                0 days 03:47:45
Name: length, dtype: object

In [379]:
minlen = routes.sort_values('length').drop_duplicates(subset = 'school_id', keep = 'first')
minlen.length.describe()


count                          280
mean     0 days 01:13:08.482142857
std      0 days 00:26:30.237352846
min                0 days 00:00:31
25%         0 days 00:54:36.250000
50%                0 days 01:12:16
75%         0 days 01:30:28.250000
max                0 days 02:49:53
Name: length, dtype: object

In [383]:
sum(minlen.length > datetime.timedelta(hours = 1))

187